In [11]:
import IPython.display
import pandas
import np_session

In [6]:
import datetime
import pathlib  
import shutil
import time
import pprint
from typing import Generator, Optional
import IPython.display

import np_session
from np_session import Session, sessions as get_sessions

problem_sessions = ('_20220825', '_20220927')
non_DOC_sessions = (
    '1172963963_604914_20220425',
    '1173189336_604914_20220426',
    '1173455023_604914_20220427',
    '1173741216_604914_20220428',
    '1182427414_607660_20220606',
    '1182628226_607660_20220607',
    '1182871514_607660_20220608',
    '1183071525_607660_20220609',
    '1177900858_608671_20220517',
    '1178173272_608671_20220518',
    '1178460518_608671_20220519',
    '1178693650_608671_20220520',
    '1174553025_608672_20220502',
    '1174790219_608672_20220503',
    '1175067685_608672_20220504',
    '1175253205_608672_20220505',
    '1185043018_608980_20220616',
    '1185035738_608980_20220616',
    '1176382912_611160_20220510',
    '1176580734_611160_20220511',
    '1176791184_611160_20220512',
    '1176989662_611160_20220513',
    '1189336645_611166_20220705',
    '1189820015_611166_20220706',
    '1190094328_611166_20220707',
    '1190290940_611166_20220708',
    '1179670730_612090_20220524',
    '1179911454_612090_20220525',
    '1180107381_612090_20220526',
    '1180266229_612090_20220527',
    '1181096406_614547_20220531',
    '1181324124_614547_20220601',
    '1181535201_614547_20220602',
    '1181731440_614547_20220603',
    '1187475832_614608_20220627',
    '1187668018_614608_20220628',
    '1187940755_614608_20220629',
    '1188138256_614608_20220630',
    '1190892228_615047_20220711',
    '1191131853_615047_20220712',
    '1191375883_615047_20220713',
    '1191631184_615047_20220714',
)
def sessions(skip_already_checked=True) -> Generator[Session, None, None]:
    for session in get_sessions('DR'):
        if any(_ in str(session) for _ in problem_sessions):
            continue
        
        yield session

def get_files_created_between(
    path: str | pathlib.Path,
    glob: str = "*",
    start: float | datetime.datetime = 0,
    end: Optional[float | datetime.datetime] = None,
) -> list[pathlib.Path]:
    path = pathlib.Path(path)
    if not path.is_dir():
        path = path.parent
    if not end:
        end = time.time()
    start = start.timestamp() if isinstance(start, datetime.datetime) else start
    end = end.timestamp() if isinstance(end, datetime.datetime) else end
    ctime = lambda x: x.stat().st_ctime
    files = (file for file in path.glob(glob) if int(start) <= ctime(file) <= end)
    return sorted(files, key=ctime)

DR_sessions: Generator[Session, None, None] = sessions()

In [7]:
session = next(DR_sessions)
session.get_missing_files()

In [8]:
session.get_missing_files()

('*_surface-image4*.png',
 '*.opto.pkl',
 '*.mapping.pkl',
 '*.behavior.pkl',
 '*.areaClassifications.csv',
 '*.fiducial.png',
 '*.overlay.png',
 '*.insertionLocation.png',
 '*.ISIregistration.npz',
 '*probelocator*insertions_in_vasculature_image_space.csv',
 '*probelocator*insertions_in_rig_image_space.csv')

Cleanup files before transfer

In [9]:
if session.project.parent.name == 'DR':
    for _ in session.npexp_path.glob('*surface-image*'):
        for replace in ('-right', '-left'):
            if replace in _.name:
                _.replace(_.parent / _.name.replace(replace, ''))
for _ in session.npexp_path.glob('*platform*.bak'):
    _.unlink()
for _ in session.npexp_path.glob('*platform*D0*json'):
    _.unlink()

In [12]:
names = session.D1.names + session.D2.names
globs = session.D1.globs + session.D2.globs
paths = session.D1.paths + session.D2.paths
types = session.D1.types + session.D2.types

print(session.npexp_path)
display(df := pandas.DataFrame(({'glob': glob, 'exists': path.name if path else None} for glob, path in zip(globs, paths)), index=names))

\\allen\programs\mindscope\workgroups\np-exp\1167400342_599657_20220329


,glob,exists
ephys_raw_data_probe_A,*_probeABC,1167400342_599657_20220329_probeABC
ephys_raw_data_probe_B,*_probeABC,1167400342_599657_20220329_probeABC
ephys_raw_data_probe_C,*_probeABC,1167400342_599657_20220329_probeABC
ephys_raw_data_probe_D,*_probeDEF,1167400342_599657_20220329_probeDEF
ephys_raw_data_probe_E,*_probeDEF,1167400342_599657_20220329_probeDEF
ephys_raw_data_probe_F,*_probeDEF,1167400342_599657_20220329_probeDEF
synchronization_data,*.sync,1167400342_599657_20220329.sync
behavior_tracking,*.behavior.mp4,1167400342_599657_20220329.behavior.mp4
beh_cam_json,*.behavior.json,1167400342_599657_20220329.behavior.json
eye_tracking,*.eye.mp4,1167400342_599657_20220329.eye.mp4


In [5]:
if not all(paths) or not all(df['exists'].values):
    raise Warning('Some files are missing - continue with upload at your own risk!')
else:
    print('All files exist')

Warning: Some files are missing - continue with upload at your own risk!

In [6]:
for _ in paths:
    if _ and str(session.id) not in _.name:
        print(f'Warning! File name does not contain session id: {_.name}')
else:
    print('All filenames contain session id')


All filenames contain session id


Get D1 & D2 `files` dicts for platform json - the manifests for uploading to LIMS

In [7]:
files = (session.D1.files | session.D2.files)

Remove surface images that don't exist: Ryan searched for missing files
already, so folders on np-exp now are as complete as possible

In [12]:
for idx, name in enumerate(names):
    if '_surface_image_' in name and paths[idx] is None:
        del files[name]

If a file or dir has already been uploaded to lims, the upload job can fail. 

Remove from the upload manifest:

In [14]:
if session.lims_path:    
    for name, path, glob in zip(names, paths, globs):
        
        if (
            (path and (file_on_lims := (session.lims_path / path.name)).exists())
            or (glob and (file_on_lims := (next(session.lims_path.rglob(glob), None))) and file_on_lims.exists())
        ):
            print(f'File or dir has already been uploaded:\n\t', file_on_lims, '\n')
            
            if name in files:
                del files[name]
                print(f'\tRemoved from upload manifest\n')

session.platform_json.files = files

File or dir has already been uploaded:
	 \\allen\programs\mindscope\production\dynamicrouting\prod0\specimen_1145398775\ecephys_session_1178693650\1178693650_608671_20220520_probeABC 

File or dir has already been uploaded:
	 \\allen\programs\mindscope\production\dynamicrouting\prod0\specimen_1145398775\ecephys_session_1178693650\1178693650_608671_20220520_probeABC 

File or dir has already been uploaded:
	 \\allen\programs\mindscope\production\dynamicrouting\prod0\specimen_1145398775\ecephys_session_1178693650\1178693650_608671_20220520_probeABC 

File or dir has already been uploaded:
	 \\allen\programs\mindscope\production\dynamicrouting\prod0\specimen_1145398775\ecephys_session_1178693650\1178693650_608671_20220520_probeDEF 

File or dir has already been uploaded:
	 \\allen\programs\mindscope\production\dynamicrouting\prod0\specimen_1145398775\ecephys_session_1178693650\1178693650_608671_20220520_probeDEF 

File or dir has already been uploaded:
	 \\allen\programs\mindscope\product

In [14]:
IPython.display.JSON(session.platform_json.path)

<IPython.core.display.JSON object>

Set up files required for transfer in scratch space

In [15]:
import datetime
import pathlib

scratch = pathlib.Path('//allen/scratch/aibstemp/svc_neuropix/lims_uploads')
scratch.mkdir(parents=True, exist_ok=True)

file_manifest = scratch / f'{session.id}_{datetime.datetime.now():%Y-%m-%d_%H%M}.txt'
file_manifest.touch(exist_ok=True)

log = file_manifest.with_suffix('.log')
log.touch(exist_ok=True)

shell_script = file_manifest.with_suffix('.sh')
shell_script.touch(exist_ok=True)

Write manifest of relative file paths, including specific sorted data files (not just
sorted probe folders)

In [16]:
src_file_list = []
sorted_files = []
for name, path in zip(names, paths):
    
    if path is None or name not in session.platform_json.files:
        continue    
    
    # specify selection of sorted data files to transfer (excl. AP continuous.dat)
    if name in tuple(f'ephys_raw_data_probe_{_}_sorted' for _ in 'ABCDEF'):
        sorted_files = session.D2.paths_sorted_data if sorted_files == [] else sorted_files
        
        #TODO LFP continuous.dat might be deleted - write symlinks to raw copy on npexp/lims
        src_file_list.extend(_ for _ in sorted_files if path in _.parents) #if path is None this should raise 
        continue
    
    src_file_list.append(path)
    
# add platform json last as a sentinel for successful transfer 
src_file_list.append(session.platform_json.path)

if any(missing := tuple(_ for _ in src_file_list if not _.exists())):
    raise FileNotFoundError('Some files do not exist: ', missing)

file_manifest.write_text(
    '\n'.join(_.relative_to(session.npexp_path).as_posix() for _ in src_file_list),
    newline='\n'
    )
print(file_manifest.read_text())

No files found for glob: \\allen\programs\mindscope\workgroups\np-exp\1179670730_612090_20220524\*_probeA*_sorted\continuous\Neuropix-PXI-???.1\continuous.dat
No files found for glob: \\allen\programs\mindscope\workgroups\np-exp\1179670730_612090_20220524\*_probeB*_sorted\continuous\Neuropix-PXI-???.1\continuous.dat
No files found for glob: \\allen\programs\mindscope\workgroups\np-exp\1179670730_612090_20220524\*_probeC*_sorted\continuous\Neuropix-PXI-???.1\continuous.dat
No files found for glob: \\allen\programs\mindscope\workgroups\np-exp\1179670730_612090_20220524\*_probeD*_sorted\continuous\Neuropix-PXI-???.1\continuous.dat
No files found for glob: \\allen\programs\mindscope\workgroups\np-exp\1179670730_612090_20220524\*_probeE*_sorted\continuous\Neuropix-PXI-???.1\continuous.dat
No files found for glob: \\allen\programs\mindscope\workgroups\np-exp\1179670730_612090_20220524\*_probeF*_sorted\continuous\Neuropix-PXI-???.1\continuous.dat


AttributeError: 'NoneType' object has no attribute 'parents'

In [36]:
rsync_cmd = f'rsync -Larv --log-file={file_manifest.with_suffix(".log").as_posix()} --files-from={file_manifest.as_posix()} {session.npexp_path.as_posix()} {np_session.INCOMING_ROOT.as_posix()}'
# -a archive mode
# -r recursive (for dirs)
# -v verbose
# -L copy the data that symlinks point to
rsync_cmd

'rsync -arv --log-file=//allen/scratch/aibstemp/svc_neuropix/lims_uploads/1226010097_2023-03-18_2253.log --files-from=//allen/scratch/aibstemp/svc_neuropix/lims_uploads/1226010097_2023-03-18_2253.txt //allen/programs/mindscope/workgroups/np-exp/1226010097_636740_20221114 //allen/programs/braintv/production/incoming/neuralcoding'

Write shell script for the transfer

In [37]:
shell_script_cmd = f"""#!/bin/bash
#SBATCH --job-name=npexp_to_incoming                        # Job name
#SBATCH --mail-type=END,FAIL                                # Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --mail-user=ben.hardcaslt@alleninstitute.org        # Where to send mail  
#SBATCH --ntasks=1                                          # Run on a single CPU
#SBATCH --mem=4gb                                           # Job memory request (per node)
#SBATCH --time=02:00:00                                     # Time limit hrs:min:sec
#SBATCH --output=npexp_to_incoming_%j.log                   # Standard output and error log
#SBATCH --partition braintv                                 # Partition used for processing
#SBATCH --tmp=100M                                          # Request the amount of space your jobs needs on /scratch/fast
 
pwd; hostname; date
 
echo 'Running rsync job on a single thread'

{rsync_cmd}
 
date
"""

shell_script.write_text(shell_script_cmd, newline='\n') 
# if writing on Windows, newline=\r\n by default, which isn't compatible with bash on linux

1241

In [38]:
import fabric
import np_config

login = np_config.fetch('/logins')['svc_neuropix']
with fabric.Connection(
        host='svc_neuropix@hpc-login',
        connect_kwargs={'password': login['password']},
    ) as ssh:
    ssh.run(f'sbatch {shell_script.as_posix()}')

Submitted batch job 8389951


In [39]:
import time

destination_file_list = [np_session.INCOMING_ROOT / _.relative_to(session.npexp_path) for _ in src_file_list]

while not all(_.exists() for _ in destination_file_list):
    print('Waiting for transfer to complete...\r', end='', flush=True)
    time.sleep(10)
    
print('\nTransfer complete')

Waiting for transfer to complete...
Transfer complete


Check platform json for upload

In [40]:
import shutil
dest_platform_json = np_session.INCOMING_ROOT / session.platform_json.path.name
shutil.copy2(session.platform_json.path, dest_platform_json)
IPython.display.JSON(np_session.INCOMING_ROOT / session.platform_json.path.name)

<IPython.core.display.JSON object>

In [25]:
np_session.write_trigger_file(session)
print('Trigger file written')

No platform json found for 1226010097 in incoming dir - lims upload job will fail when triggered: \\allen\programs\braintv\production\incoming\neuralcoding


Trigger file written


In [26]:
np_session.Session(1179670730).data_dict

{'es_id': '1179670730',
 'es': '20220524124446_704511589',
 'storage_directory': WindowsPath('//allen/programs/mindscope/production/dynamicrouting/prod0/specimen_1154273215/ecephys_session_1179670730'),
 'workflow_state': 'created',
 'date_of_acquisition': datetime.datetime(2022, 5, 24, 14, 26, 31, 876000),
 'stimulus_name': 'EPHYS_1',
 'foraging_id': '3fb9da761fcc479bb781a866340a3aa4',
 'external_specimen_name': '612090',
 'isi_experiment_id': 1158852793,
 'rig': 'NP.1',
 'operator': 'taminar',
 'project': 'DynamicRoutingSurgicalDevelopment',
 'behavior_dir': WindowsPath('//allen/programs/mindscope/production/dynamicrouting/prod0/specimen_1154273215/behavior_session_1179717742'),
 'SideDlcOutputFile': WindowsPath('//allen/programs/mindscope/production/dynamicrouting/prod0/specimen_1154273215/ecephys_session_1179670730/side_tracking/1179670730_612090_20220524.behaviorDeepCut_resnetNone_np3_side_camMar31shuffle1_1030000.h5'),
 'FaceDlcOutputFile': WindowsPath('//allen/programs/mindscope